# 04. 카운트 기반의 단어 표현(Count-based word Representation)

기본적으로 크게 두 가지 표현 방법이 있습니다.

 - 국소 표현(Local Representation) = 이산 표현(Discrete Repre-)
 - 분산 표현(Distributed Representation) = 연속 표현(Continuous Repre-)
 
국소 표현은 단순히 강아지, 귀여운, 사랑스러운 을 1, 2, 3 으로 맵핑하는 것이며,

분산 표현은 강아지와 자주 등장하는 귀여운, 사랑스러운을 같이 엮는 것입니다.

국소 표현은 주로 One-Hot Vector, N-gram, BoW 등이 있으며,
분산 표현은 Word2Vec, LSA, Glove 등이 있습니다.

## 4.2 Bag of Words(BoW)

Bag of Words는 단어들의 순서는 전혀 고려하지 않고, 출현 빈도만 집중하는 수치화 표현 방법입니다.

BoW 만드는 과정
1. 우선, 각 단어에 고유한 정수 인덱스를 부여합니다.
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

### 4.2.1. Bag of Words 예제

In [1]:
from konlpy.tag import Okt
import re

In [2]:
okt = Okt()

# 정규 표현식을 통해 온점 제거
token = re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")

# Okt 형태소 분석기를 통해 토큰화 작업 수행 후, token에 할당
token = okt.morphs(token)

In [3]:
word2index = {}
bow = []
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
        # token을 읽으면서, word2index 에 없는 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.
        
        bow.insert(len(word2index)-1, 1)
        # bow 전체에 기본값 1을 넣어줍니다.
        
    else:
        index = word2index.get(voca)
        # 재등장하는 단어의 인덱스를 받아옵니다.
        
        bow[index] = bow[index] +1
        # 재등장한 단어는 인덱스의 위치에 1을 더해줍니다.( 단어를 세어줍니다.)
        
print(word2index)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [4]:
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

### 4.2.3. CountVectorizer 클래스로 BoW 만들기

사이킷 런에서 단어의 빈도를 Count하여 Vector로 만드는 클래스가 있습니다.


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
corpus = ['you know I want your love. Because I love you.']

In [7]:
vector = CountVectorizer()

In [8]:
print(vector.fit_transform(corpus).toarray())

[[1 1 2 1 2 1]]


In [9]:
print(vector.vocabulary_)

{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


### 4.2.4. 불용어를 제거한 BoW 만들기

#### 4.2.4.1. 사용자가 직접 정의한 불용어 사용

In [10]:
text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"]) # 여기 파일 끌어와도 됨
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### 4.2.4.2. CountVectorizer에서 제공하는 자체 불용어 사용

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


#### 4.2.4.3. NLTK에서 지원하는 불용어 사용

In [12]:
from nltk.corpus import stopwords

text=["Family is not an important thing. It's everything."]
sw = stopwords.words("english")
vect = CountVectorizer(stop_words =sw)
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


## 4.3. 문서 단어 행렬(Document-Term Matrix, DTM)

간단하게 말하면 DTM은 BoW를 행렬로 만들어 놓은 것입니다.
BoW가 하나의 벡터라고 한다면 그것을 쌓아 매트릭스로 만든 것이지요.

다만 문제점이라고 하면

- 희소 표현: 매트릭스의 대부분의 값이 0이 되어서 리소스 낭비가 극심함.
- 단순 빈도 수 기반 접근: the 같은 애들이 많다고 연관성 있는 거 아님.

## 4.4. TF-IDF(Term Frequency-Inverse Document Frequency)

이놈은 말입니다. 단어의 빈도와 역 문석 빈도를 사용하여 DTM내의 각 단어들마다 중요도를 가중치로 줍니다.

그래서 DTM을 만들고, TF-IDF로 가중치를 부여합니다. 여기서

 - d: 문서
 - t: 단어
 - n: 문서의 총 개수
 
일 때, tf, df, idf로 정의할 수 있습니다.

#### 4.4.1.1 tf(d,t): 특정 문서 d에서의 특정 단어 t의 등장 횟수.

그냥 이것은 DTM을 말하는 겁니다.

#### 4.4.1.2 df(t): 특정 단어 t가 등장한 문서의 수.

각 문서, 또는 문서들에서 특정 단어 t가 몇 번 등장했는지 중요하지 않습니다. 오로지 등장 했냐 안했냐.

#### 4.4.1.3. idf(d, t): df(t)에 반비례하는 수.

idf(d, t) = log(n/1+df(t))

IDF라는 이름을 보고 DF의 역수가 아닐까 생각했다면, IDF는 DF의 역수를 취하고 싶은 것이 맞습니다. 그런데 log와 분모에 1을 더해주는 식으로 좀 이상합니다. IDF를 DF의 역수로 사용하면 문서의 수n가 커질수록 기하급수적으로 커져서 log를 취합니다. 또한 희귀 단어늗에 log를 취하지 않으면 어마어마한 가중치가 부여됩니다.

### 4.4.2. 파이썬으로 TF-IDF 직접 구현하기

In [14]:
import pandas as pd
from math import log

In [15]:
# 4개의 문서를 docs에 저장합니다.

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [19]:
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [17]:
# TF, IDF, TF-IDF 값을 구하는 함수를 구현합니다.

N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t) # 문서 d에서 단어 t가 몇 번 등장하느냐.

def idf(t): #df의 역수이니 df를 먼저 정의하고 idf를 구해줍니다.
    df = 0 # 빈 df 만들고.
    for doc in docs: # 문서의 등장 여부를 따지려는 for 문.
        df += t in doc # docs에 t가 존재하면 더해줍니다.
    return log(N/(df + 1)) # idf 식에 맞춰서 df의 역수를 취하며 +1을 취하고 log를 취해줍니다.

def tfidf(t, d):
    return tf(t,d)*idf(t) # 두개를 곱하는 tfidf.

In [18]:
# 이제 TF를 구해보겠습니다. DTM을 데이터 프레임에 저장하는 겁니다.

result = [] # 빈 객체 만들어 두고.
for i in range(N): # 문서 개수만큼 반복합니다. 여기서는 4개.
    result.append([]) # 각 문서마다 result 리스트 넣어줍니다.
    d = docs[i] # d 는 문서의 숫자쥬?
    for j in range(len(vocab)): # 고유 단어의 수만큼 반복합니다.
        t = vocab[j] # 특정 단어를 빼냅니다.
        result[-1].append(tf(t, d)) # 특정 문서에 t의 카운트를 넣습니다.

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [21]:
# 이제 DTM에 대한 IDF 값을 구해봅니다.

result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))
    
idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [23]:
# 이제 TF-IDF 행렬을 출력해봅니다.

result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))
        
tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 4.4.3. 사이킷런을 이용한 DTM과 TF-IDF 실습

In [24]:
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

In [25]:
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [26]:
# 심지어 사이킷런은 TF-IDF를 자동 계산 해주는 TfidVectorizer를 제공합니다.
# 다만 기본식에서 IDF의 로그항의 분자에 1을 더해주고, 로그항에 1을 더해주고, TF-IDF에 L2정규화를 줍니다.

from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
